In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [3]:
import sklearn
print(sklearn.__version__)

0.19.0


In [4]:
loans = pd.read_csv("C:/Users/user/Desktop/lending-club-data.csv")
# Pandas 透過使用中括號 [] 與 .iloc 
# 可以很靈活地從 data frame 中選擇想要的元素
loans.iloc[0]


C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id                                                                       1077501
member_id                                                                1296599
loan_amnt                                                                   5000
funded_amnt                                                                 5000
funded_amnt_inv                                                             4975
term                                                                   36 months
int_rate                                                                   10.65
installment                                                               162.87
grade                                                                          B
sub_grade                                                                     B2
emp_title                                                                    NaN
emp_length                                                             10+ years
home_ownership              

In [5]:
loans.bad_loans .value_counts()

0    99457
1    23150
Name: bad_loans, dtype: int64

In [6]:
# 刪除payment_inc_ratio欄位中NA的資料
# ~ --> 排除loans.payment_inc_ratio.isnull()的資料
loans = loans[~loans.payment_inc_ratio.isnull()]

In [39]:
loans[loans.payment_inc_ratio.isnull()]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none


In [7]:
model_variables = ['grade', 'home_ownership','emp_length_num', 'sub_grade','short_emp',
            'dti', 'term', 'purpose', 'int_rate', 'last_delinq_none', 'last_major_derog_none',
            'revol_util', 'total_rec_late_fee', 'payment_inc_ratio', 'bad_loans']

loans_data_relevent = loans[model_variables]


In [41]:
loans_data_relevent

,grade,home_ownership,emp_length_num,sub_grade,short_emp,dti,term,purpose,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,bad_loans
0,B,RENT,11,B2,0,27.65,36 months,credit_card,10.65,1,1,83.70,0.00,8.143500,0
1,C,RENT,1,C4,1,1.00,60 months,car,15.27,1,1,9.40,0.00,2.393200,1
2,C,RENT,11,C5,0,8.72,36 months,small_business,15.96,1,1,98.50,0.00,8.259550,0
3,C,RENT,11,C1,0,20.00,36 months,other,13.49,0,1,21.00,16.97,8.275850,0
4,A,RENT,4,A4,0,11.20,36 months,wedding,7.90,1,1,28.30,0.00,5.215330,0
5,E,RENT,10,E1,0,5.35,36 months,car,18.64,1,1,87.50,0.00,2.735750,0
6,F,OWN,5,F2,0,5.55,60 months,small_business,21.28,1,1,32.60,0.00,4.571700,1
7,B,RENT,1,B5,1,18.08,60 months,other,12.69,1,1,36.50,0.00,9.716000,1
8,C,OWN,6,C3,0,16.12,60 months,debt_consolidation,14.65,1,1,20.60,0.00,2.557500,0
9,B,OWN,11,B5,0,10.78,36 months,debt_consolidation,12.69,1,1,67.10,0.00,6.440640,0


In [8]:
loans_relevant_enconded = pd.get_dummies(loans_data_relevent)

In [43]:
loans_relevant_enconded

,emp_length_num,short_emp,dti,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,bad_loans,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
0,11,0,27.65,10.65,1,1,83.70,0.00,8.143500,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1.00,15.27,1,1,9.40,0.00,2.393200,1,...,0,0,0,0,0,0,0,0,0,0
2,11,0,8.72,15.96,1,1,98.50,0.00,8.259550,0,...,0,0,0,0,0,0,0,1,0,0
3,11,0,20.00,13.49,0,1,21.00,16.97,8.275850,0,...,0,0,0,0,0,0,1,0,0,0
4,4,0,11.20,7.90,1,1,28.30,0.00,5.215330,0,...,0,0,0,0,0,0,0,0,0,1
5,10,0,5.35,18.64,1,1,87.50,0.00,2.735750,0,...,0,0,0,0,0,0,0,0,0,0
6,5,0,5.55,21.28,1,1,32.60,0.00,4.571700,1,...,0,0,0,0,0,0,0,1,0,0
7,1,1,18.08,12.69,1,1,36.50,0.00,9.716000,1,...,0,0,0,0,0,0,1,0,0,0
8,6,0,16.12,14.65,1,1,20.60,0.00,2.557500,0,...,1,0,0,0,0,0,0,0,0,0
9,11,0,10.78,12.69,1,1,67.10,0.00,6.440640,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
# 建立training data & test data
training_features, test_features, \
training_target, test_target, = train_test_split(loans_relevant_enconded.drop(['bad_loans'], axis=1),
                                               loans_relevant_enconded['bad_loans'],
                                               test_size = .1,
                                               random_state=12)


In [31]:
training_features

,emp_length_num,short_emp,dti,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,grade_A,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
103280,3,0,18.01,12.69,1,1,91.40,0.0,15.586600,0,...,0,0,0,0,0,0,1,0,0,0
109778,2,0,11.50,10.15,1,0,58.00,0.0,3.755420,0,...,0,0,0,0,0,0,0,0,0,0
6330,5,0,14.62,22.74,0,1,87.10,0.0,6.029000,0,...,1,0,0,0,0,0,0,0,0,0
34307,1,1,14.60,13.17,0,1,48.50,0.0,3.377600,0,...,0,0,0,0,0,0,0,0,0,0
6118,5,0,18.68,6.03,1,1,6.00,0.0,12.212600,1,...,1,0,0,0,0,0,0,0,0,0
17789,3,0,4.51,10.00,0,1,18.50,0.0,13.767500,0,...,1,0,0,0,0,0,0,0,0,0
31280,2,0,13.87,11.48,1,1,42.40,0.0,9.495120,0,...,0,0,0,0,0,0,0,0,0,0
39455,6,0,26.31,20.50,1,1,51.40,0.0,11.796900,0,...,0,0,0,0,0,0,0,0,0,0
4760,8,0,2.03,11.71,1,1,1.30,0.0,8.678620,0,...,0,0,1,0,0,0,0,0,0,0
52113,4,0,14.36,15.88,0,1,76.10,0.0,7.657820,0,...,0,1,0,0,0,0,0,0,0,0


In [10]:
x_train, x_val, y_train, y_val = train_test_split(training_features, training_target,
                                                  test_size = .1,
                                                  random_state=12)


In [50]:
x_val

,emp_length_num,short_emp,dti,int_rate,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,payment_inc_ratio,grade_A,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
64176,6,0,25.38,23.63,0,1,93.6,0.0000,14.232900,0,...,0,0,0,0,0,0,0,0,0,0
7269,5,0,2.89,11.49,1,1,47.8,0.0000,5.080830,0,...,1,0,0,0,0,0,0,0,0,0
101165,0,1,11.25,13.11,1,1,53.3,0.0000,5.097500,0,...,0,0,0,0,0,0,0,0,0,0
88137,10,0,18.63,17.77,1,1,63.8,0.0000,16.216800,0,...,1,0,0,0,0,0,0,0,0,0
115081,11,0,9.32,15.61,0,0,68.2,0.0000,1.966870,0,...,0,0,0,0,0,0,0,0,0,0
78832,2,0,18.84,11.14,0,0,67.9,0.0000,8.245070,0,...,0,0,0,0,0,0,0,0,0,0
39682,7,0,17.33,25.89,0,0,43.5,0.0000,15.774200,0,...,1,0,0,0,0,0,0,0,0,0
19117,4,0,12.01,6.91,0,1,31.0,0.0000,4.625400,1,...,1,0,0,0,0,0,0,0,0,0
114466,11,0,4.94,14.16,0,0,26.3,0.0000,6.004630,0,...,1,0,0,0,0,0,0,0,0,0
92873,11,0,29.34,9.76,1,1,56.5,0.0000,6.356500,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)


C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [13]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=12)
clf_rf.fit(x_train_res, y_train_res)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [14]:
print ('Validation Results')
print (clf_rf.score(x_val, y_val))
print (recall_score(y_val, clf_rf.predict(x_val)))
print ('\nTest Results')
print (clf_rf.score(test_features, test_target))
print (recall_score(test_target, clf_rf.predict(test_features)))


Validation Results
0.800362483009
0.138195777351

Test Results
0.803278688525
0.142546718818
